# Prepare Data

Plan - Acquire - **Prepare** - Explore - Model - Deliver

## What we are doing and why:

**What:** Clean and tidy our data so that it is ready for exploration, analysis and modeling

**Why:** Set ourselves up for certainty! 

    1) Ensure that our observations will be sound:
        Validity of statistical and human observations
    2) Ensure that we will not have computational errors:
        non numerical data cells, nulls/NaNs
    3) Protect against overfitting:
        Ensure that have a split data structure prior to drawing conclusions

In [1]:
# ------------

## High level Roadmap:

#### **Input:** An aquired dataset (One Pandas Dataframe) ------> **Output:** Tidied and cleaned data split into Train,  Validate, and Test sets (Three Pandas Dataframes)

#### **Processes:** Summarize the data ---> Clean the data ---> Split the data

## Summarize

In [2]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from env import host, user, password

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings

import acquire

In [3]:
# Grab our acquired dataset:

In [4]:
df = acquire.get_titanic_data()

In [5]:
type(df)

pandas.core.frame.DataFrame

In [6]:
# take a look at our data:

In [7]:
df.shape

(891, 13)

In [8]:
df.head(5)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


In [9]:
num_cols = df.columns[[df[col].dtype != 'O' for col in df.columns]]

In [10]:
obj_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]

In [11]:
# Describe our object columns:

In [12]:
# describe object columns
for col in obj_cols:
    print(df[col].value_counts())
    print('--------------')
    print(df[col].value_counts(normalize=True, dropna=False))
    print('-------------')

male      577
female    314
Name: sex, dtype: int64
--------------
male      0.647587
female    0.352413
Name: sex, dtype: float64
-------------
S    644
C    168
Q     77
Name: embarked, dtype: int64
--------------
S      0.722783
C      0.188552
Q      0.086420
NaN    0.002245
Name: embarked, dtype: float64
-------------
Third     491
First     216
Second    184
Name: class, dtype: int64
--------------
Third     0.551066
First     0.242424
Second    0.206510
Name: class, dtype: float64
-------------
C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: deck, dtype: int64
--------------
NaN    0.772166
C      0.066218
B      0.052750
D      0.037037
E      0.035915
A      0.016835
F      0.014590
G      0.004489
Name: deck, dtype: float64
-------------
Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64
--------------
Southampton    0.722783
Cherbourg      0.188552
Queenstown     0.086420
NaN            0.002245
Name: embark_town, dtype: fl

In [13]:
# missing values:

In [14]:
missing = df.isnull().sum()
missing

passenger_id      0
survived          0
pclass            0
sex               0
age             177
sibsp             0
parch             0
fare              0
embarked          2
class             0
deck            688
embark_town       2
alone             0
dtype: int64

In [15]:
missing[missing > 0]

age            177
embarked         2
deck           688
embark_town      2
dtype: int64

#### Gather our takeaways, i.e., what we are going to do when we clean:

1. embarked == embark_town. Pick one to keep since they are dupicates.  We'll keep embarked_town for now since it is more human readable.

2. class == pclass.  They say the same thing, so let's keep the one that is numerical.

3. deck and age have many missing values.  Deck will be of no use to us with that many missing values and we will say the same of age without more insight. We will drop these columns.

4. We have just two values missing from embark_town, and we will fill these in with the most common embark_town category

5. For embark_town and sex, we will encode the values

In [16]:
# take out duplicate rows, alternatively df.drop_duplicates()
df.drop_duplicates(inplace=True)

In [17]:
df.shape

(891, 13)

In [18]:
# drop our columns:
columns_to_drop = ['deck', 'age', 'embarked', 'class']

In [19]:
df = df.drop(columns=columns_to_drop)

## Clean

In [20]:
# drop duplicates...run just in case


In [21]:
# drop columns with too many missing to have any value right now


We could fill embark_town with most common value, 'Southampton', by hard-coding the value using the fillna() function, as below. Or we could use an imputer. We will demonstrate the imputer *after* the train-validate-test split. 

In [22]:
df['embark_town'] = df.embark_town.fillna(value='Southampton')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  891 non-null    int64  
 1   survived      891 non-null    int64  
 2   pclass        891 non-null    int64  
 3   sex           891 non-null    object 
 4   sibsp         891 non-null    int64  
 5   parch         891 non-null    int64  
 6   fare          891 non-null    float64
 7   embark_town   891 non-null    object 
 8   alone         891 non-null    int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 69.6+ KB


In [24]:
dummy_df = pd.get_dummies(df[['sex', 'embark_town']], dummy_na=False, drop_first=[True, True])

In [25]:
# Let's make that into a function so we can repeat it all easily in one step.

In [26]:
# rows: r0ws
# cols: co1s
df = pd.concat([df, dummy_df], axis=1)
df

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,0,3,male,1,0,7.2500,Southampton,0,1,0,1
1,1,1,1,female,1,0,71.2833,Cherbourg,0,0,0,0
2,2,1,3,female,0,0,7.9250,Southampton,1,0,0,1
3,3,1,1,female,1,0,53.1000,Southampton,0,0,0,1
4,4,0,3,male,0,0,8.0500,Southampton,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,0,0,13.0000,Southampton,1,1,0,1
887,887,1,1,female,0,0,30.0000,Southampton,1,0,0,1
888,888,0,3,female,1,2,23.4500,Southampton,0,0,0,1
889,889,1,1,male,0,0,30.0000,Cherbourg,1,1,0,0


In [27]:
df = acquire.get_titanic_data()

Create a function to perform these steps when we need to reproduce our dataset. 

In [36]:
def clean_titanic_data(df):
    '''
    takes in a datafram of the titanic dataset as it is acquired and return a cleaned
    dataframe arguments: df: a panda DataFrame with the expected featrure names and columns
    return: clean_df: a dataframe with the cleaning operations performed on it
    '''
    df = df.drop_duplicates()
    df = df.drop(columns=['deck', 'embarked', 'class', 'age'])
    df['embark_town'] = df.embark_town.fillna(value='Southampton')
    dummy_df = pd.get_dummies(df[['sex', 'embark_town']], drop_first=True)
    df = pd.concat([df, dummy_df], axis=1)
    return df.drop(columns=['sex', 'embark_town'])

In [39]:
cleaned_df_v0 = clean_titanic_data(df)
cleaned_df_v0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   passenger_id             891 non-null    int64  
 1   survived                 891 non-null    int64  
 2   pclass                   891 non-null    int64  
 3   sibsp                    891 non-null    int64  
 4   parch                    891 non-null    int64  
 5   fare                     891 non-null    float64
 6   alone                    891 non-null    int64  
 7   sex_male                 891 non-null    uint8  
 8   embark_town_Queenstown   891 non-null    uint8  
 9   embark_town_Southampton  891 non-null    uint8  
dtypes: float64(1), int64(6), uint8(3)
memory usage: 58.3 KB


## Train, Validate, Test Split

In [43]:
# 20% test, 80% train_validate
# then of the 80% train_validate: 30% validate, 70% train. 
train, test = train_test_split(cleaned_df_v0, test_size=0.2, random_state=1349, stratify=cleaned_df_v0.survived)
train, validate = train_test_split(train, test_size=0.7, random_state=1349, stratify=train.survived)

## Option for Missing Values: Impute

We can impute values using the mean, median, mode (most frequent), or a constant value. We will use sklearn.imputer.SimpleImputer to do this.  

1. Create the imputer object, selecting the strategy used to impute (mean, median or mode (strategy = 'most_frequent'). 
2. Fit to train. This means compute the mean, median, or most_frequent (i.e. mode) for each of the columns that will be imputed. Store that value in the imputer object. 
3. Transform train: fill missing values in train dataset with that value identified
4. Transform test: fill missing values with that value identified

1. Create the `SimpleImputer` object, which we will store in the variable `imputer`. In the creation of the object, we will specify the strategy to use (`mean`, `median`, `most_frequent`). Essentially, this is creating the instructions and assigning them to a variable we will reference.  

In [44]:
imputer = SimpleImputer(strategy='most_frequent', missing_values=None)

In [45]:
type(imputer)

sklearn.impute._base.SimpleImputer

2. `Fit` the imputer to the columns in the training df.  This means that the imputer will determine the `most_frequent` value, or other value depending on the `strategy` called, for each column.   

In [ ]:
imputer = imputer.fit(train[['embark_town']])

3. It will store that value in the imputer object to use upon calling `transform.` We will call `transform` on each of our samples to fill any missing values.  

In [ ]:
train[['embark_towns']] = imputer.transform(train[['embark_town']])
validate[['embark_towns']] = imputer.transform(validate[['embark_town']])
test[['embark_towns']] = imputer.transform(test[['embark_town']])

Create a function that will run through all of these steps, when I provide a train and test dataframe, a strategy, and a list of columns. 

In [58]:
def impute_mode(train, validate, test):
    '''
    impute mode for embark_town
    '''
    imputer = SimpleImputer(strategy='most_frequent', missing_values=None)
    train[['embark_town']] = imputer.fit_transform(train[['embark_town']])
    validate[['embark_town']] = imputer.transform(validate[['embark_town']])
    test[['embark_town']] = imputer.transform(test[['embark_town']])
    return train, validate, test

Blend the clean, split and impute functions into a single prep_data() function. 

In [55]:
def prep_titanic_data(df):
    '''
    takes in a dataframe of the titanic dataset as it is acquired and returns a cleaned dataframe
    arguments: df: a pandas DataFrame with the expected feature names and columns
    return: train, test, split: three dataframes with the cleaning operations performed on them
    '''
    df = df.drop_duplicates()
    df = df.drop(columns=['deck', 'embarked', 'class', 'age', 'passenger_id'])
    train, test = train_test_split(df, test_size=0.2, random_state=1349, stratify=df.survived)
    train, validate = train_test_split(train, train_size=0.7, random_state=1349, stratify=train.survived)
#     print(train.info())
#     return train, validate, test
    train, validate, test = impute_mode(train, validate, test)
    dummy_train = pd.get_dummies(train[['sex', 'embark_town']], drop_first=[True,True])
    dummy_validate = pd.get_dummies(validate[['sex', 'embark_town']], drop_first=[True,True])
    dummy_test = pd.get_dummies(test[['sex', 'embark_town']], drop_first=[True,True])
    train = pd.concat([train, dummy_train], axis=1)
    validate = pd.concat([validate, dummy_validate], axis=1)
    test = pd.concat([test, dummy_test], axis=1)
    train = train.drop(columns=['sex', 'embark_town'])
    validate = validate.drop(columns=['sex', 'embark_town'])
    test = test.drop(columns=['sex', 'embark_town'])
    return train, validate, test

In [59]:
df = acquire.get_titanic_data()

In [60]:
train, validate, test = prep_titanic_data(df)

<ipython-input-58-e94e11ddf483>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['embark_town']] = imputer.transform(test[['embark_town']])
/opt/homebrew/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 474 to 94
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 498 non-null    int64  
 1   pclass                   498 non-null    int64  
 2   sibsp                    498 non-null    int64  
 3   parch                    498 non-null    int64  
 4   fare                     498 non-null    float64
 5   alone                    498 non-null    int64  
 6   sex_male                 498 non-null    uint8  
 7   embark_town_Queenstown   498 non-null    uint8  
 8   embark_town_Southampton  498 non-null    uint8  
dtypes: float64(1), int64(5), uint8(3)
memory usage: 28.7 KB


In [63]:
validate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 569 to 845
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 214 non-null    int64  
 1   pclass                   214 non-null    int64  
 2   sibsp                    214 non-null    int64  
 3   parch                    214 non-null    int64  
 4   fare                     214 non-null    float64
 5   alone                    214 non-null    int64  
 6   sex_male                 214 non-null    uint8  
 7   embark_town_Queenstown   214 non-null    uint8  
 8   embark_town_Southampton  214 non-null    uint8  
dtypes: float64(1), int64(5), uint8(3)
memory usage: 12.3 KB


In [65]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 691 to 799
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 179 non-null    int64  
 1   pclass                   179 non-null    int64  
 2   sibsp                    179 non-null    int64  
 3   parch                    179 non-null    int64  
 4   fare                     179 non-null    float64
 5   alone                    179 non-null    int64  
 6   sex_male                 179 non-null    uint8  
 7   embark_town_Queenstown   179 non-null    uint8  
 8   embark_town_Southampton  179 non-null    uint8  
dtypes: float64(1), int64(5), uint8(3)
memory usage: 10.3 KB


In [70]:
train, validate, test = impute_mode()

TypeError: impute_mode() missing 3 required positional arguments: 'train', 'validate', and 'test'

In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 474 to 94
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 498 non-null    int64  
 1   pclass                   498 non-null    int64  
 2   sibsp                    498 non-null    int64  
 3   parch                    498 non-null    int64  
 4   fare                     498 non-null    float64
 5   alone                    498 non-null    int64  
 6   sex_male                 498 non-null    uint8  
 7   embark_town_Queenstown   498 non-null    uint8  
 8   embark_town_Southampton  498 non-null    uint8  
dtypes: float64(1), int64(5), uint8(3)
memory usage: 28.7 KB


In [69]:
train.head()

,survived,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
474,0,3,0,0,9.8375,1,0,0,1
370,1,1,1,0,55.4417,0,1,0,0
573,1,3,0,0,7.7500,1,0,1,0
110,0,1,0,0,52.0000,1,1,0,1
167,0,3,1,4,27.9000,0,0,0,1


## Exercises

The end product of this exercise should be the specified functions in a python script named `prepare.py`.
Do these in your `classification_exercises.ipynb` first, then transfer to the prepare.py file. 

This work should all be saved in your local `classification-exercises` repo. Then add, commit, and push your changes.

Using the Iris Data:  

1. Use the function defined in `acquire.py` to load the iris data.  

1. Drop the `species_id` and `measurement_id` columns.  

1. Rename the `species_name` column to just `species`.  

1. Create dummy variables of the species name. 

1. Create a function named `prep_iris` that accepts the untransformed iris data, and returns the data with the transformations above applied.  